In [2]:
!pip install trl sacrebleu

In [3]:
import os, json, random
from pathlib import Path
from typing import Dict, Any, List

import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, PeftModel
import trl
from trl import SFTTrainer, SFTConfig
import sacrebleu

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

os.environ.setdefault("PYTORCH_ENABLE_MPS_FALLBACK", "1")
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print("device:", device)
print("torch:", torch.__version__)
print("trl:", trl.__version__)


device: cuda
torch: 2.9.0+cu126
trl: 0.26.1


In [6]:
import json
from pathlib import Path
from datasets import Dataset

DATA_PATH = Path("train_data.json")

# 1. 데이터 로드
text = DATA_PATH.read_text(encoding="utf-8").strip()

if not text:
    raise RuntimeError("train_data.json이 비어 있습니다.")

raw_data = []

try:
    # 한 줄씩 읽기 (JSONL) 시도
    for line in text.splitlines():
        if line.strip():
            raw_data.append(json.loads(line))
except json.JSONDecodeError:
    raw_data = json.loads(text)

# 2. 데이터 정제 함수
def normalize_item(item: dict) -> dict:
    # idiom 딕셔너리에서 term과 meaning을 추출하여 상위 레벨로 올림
    idiom_data = item.get("idiom", {})

    return {
        "instruction": item["instruction"].strip(),
        "input": item["input"].strip(),
        "output": item["output"].strip(),
        "term": idiom_data.get("term", "").strip(),
        "meaning": idiom_data.get("meaning", "").strip()
    }

# 3. 데이터셋 생성
norm = [normalize_item(x) for x in raw_data]
ds = Dataset.from_list(norm)

print(f"✅ 전체 학습 데이터 개수: {len(ds)}")

print("예시 데이터 항목 (첫 번째):")
if len(ds) > 0:
    print(ds[0])

✅ 전체 학습 데이터 개수: 600
예시 데이터 항목 (첫 번째):
{'instruction': "Don't translate it in Korean, but translate it according to Korean culture", 'input': 'I actually saw him at the mall yesterday, no cap.', 'output': '나 어제 쇼핑몰에서 걔 진짜로 봤다니까, 구라 아님.', 'term': 'No cap', 'meaning': "진짜야, 거짓말 아니야 ('Cap'은 거짓말을 뜻함)"}


In [8]:
from typing import Dict, Any

SYSTEM_PROMPT = """
You are a professional translator who specializes in translating English slang and memes into Korean internet slang and trendy expressions.
Your goal is to make the translation sound like a "close friend" or a "Korean netizen" speaking.

**CRITICAL RULES:**
1. **Never use polite language (Honorifics/Jon-dae-mal).** Use ONLY casual speech (Banmal).
2. Do not translate literally. Use Korean slang, memes, and community vibes aggressively.
3. If the original text is sarcastic or rude, preserve that tone perfectly.
4. Output ONLY the Korean translation. No explanations.
"""

def build_prompt(example: Dict[str, Any]) -> str:
    ref = ""
    # term이나 meaning이 있는 경우에만 참고 정보를 추가
    if example.get("term") or example.get("meaning"):
        ref = f"[Note]\n- Expression: {example.get('term','')}\n- Meaning: {example.get('meaning','')}\n\n"

    user = (
        f"지시사항: {example['instruction']}\n\n"
        f"{ref}"
        f"입력 문장: {example['input']}\n"
        f"출력 조건: 한국어 번역 1개만 출력"
    )

    return (
        "### System:\n"
        f"{SYSTEM_PROMPT.strip()}\n\n"
        "### User:\n"
        f"{user}\n\n"
        "### Assistant:\n"
    )

def to_prompt_completion(ex: Dict[str, Any]) -> Dict[str, str]:
    return {"prompt": build_prompt(ex), "completion": ex["output"]}

# [수정됨] ds가 Dataset 객체이므로 ["train"] 없이 바로 column_names에 접근합니다.
pc = ds.map(to_prompt_completion, remove_columns=ds.column_names)

# [수정됨] 결과 확인 시에도 ["train"] 키 없이 인덱스로 바로 접근합니다.
print(pc[0]["prompt"][:600])
print("completion:", pc[0]["completion"])

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

### System:
You are a professional translator who specializes in translating English slang and memes into Korean internet slang and trendy expressions.
Your goal is to make the translation sound like a "close friend" or a "Korean netizen" speaking.

**CRITICAL RULES:**
1. **Never use polite language (Honorifics/Jon-dae-mal).** Use ONLY casual speech (Banmal).
2. Do not translate literally. Use Korean slang, memes, and community vibes aggressively.
3. If the original text is sarcastic or rude, preserve that tone perfectly.
4. Output ONLY the Korean translation. No explanations.

### User:
지시사항:
completion: 나 어제 쇼핑몰에서 걔 진짜로 봤다니까, 구라 아님.


In [11]:
from google.colab import drive
drive.mount('/content/drive')

OUTPUT_DIR = "/content/drive/MyDrive/V3_hyperclova-translator-mps"
MODEL_ID = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"

# 1. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer.padding_side = "right"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,  # MPS 안정성
    trust_remote_code=True,
    device_map="auto"
)
base_model.config.use_cache = False
print("✅ 모델 로드 완료")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 모델 로드 완료


In [17]:
preferred_targets = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

def guess_target_modules(model) -> List[str]:
    names = set()
    for n,_ in model.named_modules():
        names.add(n.split(".")[-1])
    return [t for t in preferred_targets if t in names]

targets = guess_target_modules(base_model)
if not targets:
    targets = "all-linear"
    print("⚠️ target_modules 자동 탐지 실패 → all-linear 사용")
else:
    print("✅ target_modules:", targets)

peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=targets,
    bias="none",
    task_type="CAUSAL_LM",
)

training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=50,
    eval_strategy="no",
    #eval_steps=50,
    save_strategy="steps",
    save_steps=200,
    bf16=False,
    fp16=True,
    optim ="adamw_torch",
    report_to="none",
    packing=False,
    seed=SEED,
    completion_only_loss=True,
    dataset_text_field="text",
)

trainer = SFTTrainer(
    model=base_model,
    args=training_args,
    train_dataset=pc,
    peft_config=peft_config,
)

print("✅ trainer 준비 완료")


✅ target_modules: ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/600 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/600 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/600 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ trainer 준비 완료


In [18]:
print("🚀 학습 시작")
trainer.train()
print("✅ 학습 종료")

trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"🎉 저장 완료: {OUTPUT_DIR}")


🚀 학습 시작


Step,Training Loss
50,2.799500
100,1.813600
150,1.603400
200,0.940300
250,0.652100
300,0.429000
350,0.163900
400,0.136800
450,0.069200
500,0.026900


✅ 학습 종료
🎉 저장 완료: /content/drive/MyDrive/V3_hyperclova-translator-mps


In [ ]:
import shutil
import os

source = '/content/V3_hyperclova-translator-mps' # 옮길 폴더 이름
destination = '/content/drive/MyDrive/my_lab_folder' # 드라이브 내 저장할 경로

# 폴더 이동 실행
try:
    shutil.move(source, destination)
    print(f"이동 완료: {destination}")
except Exception as e:
    print(f"오류 발생: {e}")

이동 완료: /content/drive/MyDrive/my_lab_folder
